Деревья классификации
====================

* Полезные ссылки<br>
  * [бэггинг, случайный лес](https://habr.com/ru/company/ods/blog/324402/).<br>
  * [Градиентный бустинг](https://habr.com/ru/company/ods/blog/327250/).<br>
  * [Классификация, деревья решений и метод ближайших соседей](https://habr.com/ru/company/ods/blog/322534/).<br>

In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm, chi2_contingency
import statsmodels.api as sm
from numpy import linspace,hstack
from pylab import plot,show,hist
import pydot
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

#Стандартизация данных
from sklearn import preprocessing

#Построение диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

#Деревья решений для задачи классификации
from sklearn.tree import DecisionTreeClassifier

#Деревья решений для задач регрессии 
from sklearn.ensemble import RandomForestClassifier

#Калибровка деревьев решений
from sklearn.calibration import CalibratedClassifierCV

os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


#df = pd.read_csv('AmesHousing.txt', sep="\t", header = 0, index_col=False)
#df = pd.read_csv('town_1959_2.csv', header = 0,)
#df = pd.read_csv('swiss_bank_notes.csv', index_col=0)
#df = pd.read_csv('beverage_r.csv', sep=";", index_col='numb.obs')
#df = pd.read_csv('Protein Consumption in Europe.csv', sep=';', decimal=',', index_col='Country')
#df = pd.read_csv('assess.dat', sep='\t', index_col='NAME')
#df = pd.read_csv('Albuquerque Home Prices_data.txt', sep='\t')
#df = pd.read_csv('agedeath.dat.txt', sep='\s+', header=None, names=['group', 'age', 'index'])
#df = pd.read_csv('interference.csv')
#df = pd.read_csv('diamond.dat', header=None, sep='\s+', names=['weight', 'price'])
#df = pd.read_csv('Credit.csv', sep=';', encoding='cp1251')
#df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
#df = pd.read_csv('Wine.txt', sep='\t', header=0)
#df = pd.read_csv('monthly-car-sales-in-quebec-1960.csv', sep=';', header=0, parse_dates=[0])
#df = pd.read_csv('stickleback.csv', sep=';', decimal=',')
df = pd.read_csv('Swiss Fertility.csv', sep=';', decimal=',', index_col=0)

In [2]:
def two_histograms(x, y):
    """
    Функция, которая построит две гистограммы на одной картинке.
    Дополнительно пунктирными линиями указываются средние значения выборок.
    x: вектор pd.Series,
    y: вектор pd.Series
    """
    x.hist(alpha=0.5, weights=[1./len(x)]*len(x))
    y.hist(alpha=0.5, weights=[1./len(y)]*len(y))
    plt.axvline(x.mean(), color='red', alpha=0.8, linestyle='dashed')
    plt.axvline(y.mean(), color='blue', alpha=0.8, linestyle='dashed')
    plt.legend([x.name, y.name])
    
def regression_coef(model, X, y):
    """
    Функция для определения статистической значимости регрессионных коэффициентов
    """
    coef = pd.DataFrame(zip(['intercept'] + X.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
    X1 = np.append(np.ones((len(X),1)), X, axis=1)
    b = np.append(model.intercept_, model.coef_)
    MSE = np.sum((model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    var_b = MSE * (np.linalg.inv(np.dot(X1.T, X1)).diagonal())
    sd_b = np.sqrt(var_b)
    t = b / sd_b
    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (len(X1) - 1))) for i in t]
    return coef


def scale_features(df):
    """
    Функция для стандартизации переменных в датафрейме
    """
    scaled = preprocessing.StandardScaler().fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    return scaled

In [ ]:
#Стандартизация позволяет сделать вес важных переменных соизмеримым. Min=0(-1), max=1. ИЛИ Z
#Z-Стандартизация: преобразование в тип, где М=0, sd = 1. Правило одной, двух и трех "сигм"
#Z=(Xинд-М)/sd Пример: по таблице Z, где Хсред=150, sd=8, превышать Xинд будет 0.5z или 30%
#Z=(Xсред-M)/se =(18,5-20)/0.5 = -3. Вероятность получить такой результат p = 0.0027

#Если в БД нет единой метрики, то стандартизируем данные
norm = preprocessing.StandardScaler()
norm.fit(df)
X = norm.transform(df)

In [ ]:
#Задача распознавания наименований или порядков через деревья классификации. И чисел через регрессию
#Помимо внутренних параметров (заданных изначально), есть еще внешние (задаваемые аналитиком)
#Выбор модели с помощью обучающей/тестовой выборок через наименьшую среднюю квадратичную ошибку 
#Критерий качества Q - сумма модулей ошибок или сумма квадратов ошибок или процент ошибок и т.д.
#Валидация - метод проверки выбранной модели на ее адекватность
#Регуляризация - инструмент проверки моделей

In [ ]:
#CART - Classification and regression trees
#деление матрицы прямыми\гиперплоскостями, чтобы в ограниченных областях доминировали схожие объекты
#Узел(node) - множество, которое расщепляется. Родительский, потомок, конечный. 
#Пороговое значение - эталон для сравнения
#Ограничения задаются оператором. На кол-во слоев, на свойство потомков, на родителя, на правила остановки 
#Чистота - порядок разделения выборки на части, в каждой из которых "загрязнение" данных меньше
#Критерий загразненности(вероятность принадлежать к классу P) измеряется Энтропией, Индексом Джини или Ошибкой Классификации
#Энтропия H1 = -СуммаP*log2P. Индекс Джини H2 = 1-СуммаP**2 = СуммаP*(1-P). Ошибка Классификации H3 = 1-maxP
#Дельта H - вклад переменной в очищение. Считаем суммы для каждой и получаем информативность переменной

In [ ]:
from sklearn.model_selection import train_test_split

#Расщепление на обучающую и тестовую выборки

X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    # доля объёма тестового множества
                                                    test_size=0.2)

In [ ]:
#Задача кредитного скоринга
# Правильный ответ записываем в вектор y
y = df[u'кредит']
# Удаляем колонку с правильным ответом
X = df.drop(u'кредит', axis=1)

from sklearn.tree import DecisionTreeClassifier

#Инициализируем и обучаем модель
model = DecisionTreeClassifier(random_state=42,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # максимальная глубина дерева
                               max_depth=5,
                               # минимальное число элементов в узле для разбиения (может быть долей)
                               min_samples_split=5,
                               # минимальное число элементов в листе (может быть долей)
                               min_samples_leaf=5,
                               # минимальное значение дельты impurity
                               # min_impurity_decrease=0,
                               # веса для классов (можно дополнительно штрафовать за ошибку в нужных классах).
                               # поддерживает опцию 'balanced'.
                               class_weight=None
                               
                              )

model.fit(X, y)

#Для интерпретации получившейся модели изображаем её в виде дерева предикатов (решающих правил)

export_graphviz(model,
                out_file='tree.dot',
                #задать названия фич
                #feature_names=X.columns,
                class_names=None,
                #показывать названия полей у численных значений внутри узла
                label='all',
                #раскрашивать узлы в цвет преобладающего класса
                filled=True,
                #показывать значение impurity для каждого узла
                impurity=True,
                #показывать номера узлов
                node_ids=True,
                #Показывать доли каждого класса в узлах (а не количество)
                proportion=True,
                #Повернуть дерево на 90 градусов (вертикальная ориентация)
                rotate=True,
                #Число точек после запятой для отображаемых дробей
                #precision=3
               )

#Преобразуем файл .dot в .png
#node - номер узла, X[1]<=1.5 правило расщепления, gini, samples-доля наблюдений попавших в узел, p-value (p0, pX)
(graph,) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')
Image("tree.png")

#Модель позволяет оценить ценность (importance) и эффективность каждой фичи, считая для каждой из сумму дельты H 
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', 
            ascending=False
            )

#Метод predict позволяет получить предсказания классов для входного списка элементов (подаём на вход матрицу)
# Предсказание класса для новых элементов
new_item = [1, 1, 1, 1]
model.predict([new_item])

In [ ]:
#Расщепление на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    # доля объёма тестового множества
                                                    test_size=0.2)
#обучаем модель
model.fit(X_train, y_train)

#Строим предсказание модели на тестовом множестве
y_pred = model.predict(X_test)

#Оценка качества классификатора: доля совпавших ответов в y_pred и y_test, или считаем точность и полноту
#Если доля в обучающем выше тестового, означает переобученность модели. Нужно упрощать модель
#Матрица ошибок  𝐶=(𝑐𝑖,𝑗) , где  𝑐𝑖,𝑗 количество элементов класса 𝑖 , которым классификатор присвоил класс 𝑗 
#Точность(precision) - доля правильно классифицированных объектов в найденных классификатором. 
#Полнота(recall) - доля этих объектов НА САМОМ ДЕЛЕ
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

#Гармоническое среднее F1 = 2*точность*полнота/(точность+полнота). Считается с помощью classification_report
print(metrics.classification_report(y_pred, y_test))

In [ ]:
#Деревья решений для задач регрессии (отклик не дискретный, а непрерывный). Методы схожы с деревом классификации
#Предпочтительнии линейной регрессии, когда зависимость не линейная :)
#В этом случа Дельта H = сумма квадратов ошибок
#Prune (обрезание) - очистка от узлов, которые не нужны, через добавление третьей выборки (валидации)

#Случайный лес. Ключевые параметры:
#ntree - число деревьев(в начале макс, потом сокращать), mtry - число переменных в выборке (M**0.5)
#sampsize - число наблюдений в подвыборке(0.632*N для декорреляции), nodesize - мин. число наблюдений в узле (10) 
#replace - подвыборка с  возвращением или без
#out-of-bag - неиспользуемая при обучении часть выборки, используется в качестве предварительного теста модели

#Если несколько классов, но хочется сделать классификацию строго бинарной, то разбиваем на группы ДА и НЕТ
df['Desired1(3)'] = df['Desired1(3)'].replace(0, 1)


model = RandomForestClassifier(random_state=42, #зерно датчика случайных чисел
                               #число деревьев в лесу
                               n_estimators=30,
                               #функция для дельта H, impurity ('gini' или 'entropy')
                               criterion='gini',
                               #Макс число слоев
                               max_depth=5,
                               #Вычислять out-of-bag ошибку
                               oob_score=True,
                               #использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               #веса классов для балансировки выборки для обучения
                               class_weight=None
                               
                              )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(metrics.classification_report(y_pred, y_test))

print('Out-of-bag score: {0}'.format(model.oob_score_)) 

pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
#Вариант 2 Случайного леса

#Данные для предсказания
X_to_be_predicted = df[df.Data.isnull()]
X_to_be_predicted = X_to_be_predicted.drop(['data'], axis = 1)
# X_to_be_predicted[X_to_be_predicted.Age.isnull()]
# X_to_be_predicted.dropna(inplace = True) # 417 x 27
#Training data
train_data = df
train_data = train_data.dropna()
feature_train = train_data['data']
label_train = train_data.drop(['data'], axis = 1)


clf = RandomForestClassifier(criterion='entropy',
                                n_estimators=700,
                                min_samples_split=10,
                                min_samples_leaf=1,
                                max_features='auto',
                                oob_score=True,
                                random_state=1,
                                n_jobs=-1
                            )
x_train, x_test, y_train, y_test = train_test_split(label_train, feature_train, test_size=0.2)

clf.fit(x_train, np.ravel(y_train))

print("RF Accuracy: "+repr(round(clf.score(x_test, y_test) * 100, 2)) + "%")

result_rf=cross_val_score(clf,x_train,y_train,cv=10,scoring='accuracy')

print('The cross validated score for Random forest is:',round(result_rf.mean()*100,2))

y_pred = cross_val_predict(clf,x_train,y_train,cv=10)

sns.heatmap(confusion_matrix(y_train,y_pred),annot=True,fmt='3.0f',cmap="summer")

plt.title('Confusion_matrix for RF', y=1.05, size=15)

In [ ]:
#Приемы улучшения классификаторов: stacking, bagging, boosting
#Stacking(предсказание на базе предсказаний)
#Bagging(усредненное мнение всех моделей), он же случайный лес. Чтобы избежать колинеарности, выборки собираются рандомно
#Boosting - обучение на основе ошибок предыдущего классификатора (улучшением слабого классификатора)

In [ ]:
#GBM - Gradient Boosting Machine. Остановка бустинга, когда очередные циклы перстают улучшать модель
#Сумма квадратов ошибок Zi = -2*(Yi - f(Xi))
#Метод максимального правдоподобия. Предполагаем наиболее вероятное событие. Критерий качества = P**A*(1-P)**(n-A)
#Критерии качества Гаусса и Лапласа универсальны. Для двух классов - биномиальное, для большего - мультиноминальное
#При временном промежутке - распределение Пуассона

columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?')

# Удаляем колонку education (потому что есть уже закодированная колонка education-num)
df = df.drop('education', axis=1)

# Кодируем отклик в бинарные значения
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})

# удаляем строки с NA значениями
df = df.dropna()

test = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
test = test.drop('education', axis=1)
test['income'] = test['income'].map({' <=50K.': 0, ' >50K.': 1})
test = test.dropna()

#Распределение классов в отклике
df['income'].value_counts(normalize=True)

#Разбиваем дату. Бинаризуем категориальные признаки (one-hot encoding).
X_train = pd.get_dummies(df).drop('income', axis=1)
y_train = df['income']

X_test = pd.get_dummies(test).drop('income', axis=1)
y_test = test['income']

#В тестовой выборке не хватает одной колонки 
print(len(X_train.columns))
print(len(X_test.columns))

#Приводим множество названий колонок к типу set, находим разность двух множеств: Голландии нет в колонке native-county 
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

#Добавляем недостающую колонку. Смотрим, стоит ли склеивать отдельные переменные в более крупные классы
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

#Проверяем совпадение колонок (если да, то True)
all(X_train.columns == X_test.columns)

#Создаем и обучаем модель
model = GradientBoostingClassifier(random_state=42,
                                   # Число деревьев
                                   n_estimators=500,
                                   #загрязнение измеряем “mse”, “mae” или “friedman_mse” (mse с улучшениями)  
                                   criterion='friedman_mse', 
                                   #Максимальная глубина каждого дерева
                                   #критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’
                                   #‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.0, 
                                   # Устарело
                                   min_impurity_split=None,
                                   # число узлов в дереве
                                   max_depth=5,
                                   #минимальное число наблюдений в потомке
                                   min_samples_leaf=5, 
                                   #минимальное число наблюдений в родителе
                                   min_samples_split=10,
                                   #Параметр, уменьшающий переобучение, являющемся весом отдельного дерева (меньше лучше)
                                   learning_rate=0.01                                   
                                   )

model.fit(X_train, y_train)

#Строим предсказание (обучающая, тестовая)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

#Смотрим точность(ошибки 1 и 2го рода)
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

#Cмотрим важность признаков
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

#Калибровка (интерпретация вероятности)
from sklearn.calibration import CalibratedClassifierCV
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')
# method : ‘sigmoid’ or ‘isotonic’

# Calibrate probabilities
model_sigmoid.fit(X_train, y_train)

# View calibrated probabilities
model_sigmoid.predict_proba(X_test)[0:11, :]

In [ ]:
#XGBoost - множество деревьев регрессии. q - структура дерева. 
#W - вес узла, набор правил попадения наблюдений в конечный узел. 
#T - количество конечных узлов, для избежание переобучения. Автоматом определяется гаммой
#L - критерий качества XGBoost состоит из традиционного(Q)+регуляризация(1/2*гамма*T). Влияет на изменения критерия чистоты

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import classification_report
import seaborn as sns
sns.set(font_scale = 1.5)
from sklearn.grid_search import GridSearchCV


columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?')
# Удаляем колонку education (потому что есть уже закодированная колонка education-num)
df = df.drop('education', axis=1)
# Кодируем отклик в бинарные значения
df['income'] = df['income'].map({' <=50K': 0, ' >50K': 1})
# удаляем строки с NA значениями
df = df.dropna()

test = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
test = test.drop('education', axis=1)
test['income'] = test['income'].map({' <=50K.': 0, ' >50K.': 1})
test = test.dropna()

X_train = pd.get_dummies(df).drop('income', axis=1)
y_train = df['income']
X_test = pd.get_dummies(test).drop('income', axis=1)
y_test = test['income']

columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)


#Создаем модель. В выборе параметров помогают grid_search и валидация
model = XGBClassifier(seed=42, 
                      booster=gbtree, #выбор деревьев gbtree или линейных gblinear
                      silent=True, #НЕ вывод промежуточных результатов
                      n_estimators=100, #Предельное число деревьев
                      learning_rate=0.02, #скорость обучения
                      min_child_weight=5, #минимальные веса в узле-потомке
                      max_leaf_nodes=6, #Макс. значение конечных узлов в дереве
                      max_depth=5, #Максимальное число слоев дерева
                      gamma=0.05, #Запрещает расщепление, если загрязнение уменьшилось меньше чем на гамму
                      subsample=0.7, #Доля наблюдений, попадающих в случайную подвыборку
                      reg_lambda=0, #Критерий качества, сумма квадратов, mse 
                      reg_alpha=1 #Критерий качества, сумма модулей, mae                      
                     )


model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
print (classification_report(y_train, y_pred_train))
y_pred = model.predict(X_test)
print (classification_report(y_test, y_pred))

xgb.plot_importance(model)
xgb.plot_importance(model, max_num_features = 30)
            
#В выборе параметров помогают grid_search и валидация
#Оптимально перебирать параметры последовательно по одному (но наилучшее решение можно и не найти)

smart_xgboost = GridSearchCV(cv=5, #Параметр разделения для кросс-валидации (фолды)
                   error_score='raise',
                   estimator=XGBClassifier( #Задаем оценку = XGBClassifier
                               base_score=0.5, #Задаем параметры, которые не собираемся оптимизировать
                               colsample_bylevel=1, 
                               colsample_bytree=0.8,
                               gamma=0, 
                               max_delta_step=0,
                               missing=None, 
                               nthread=-1,
                               objective='binary:logistic', #Критерий кач-ва при обучении. здесь 2, больше-softmax, sofprob
                               num_class=3, #Дополнительно задачем число классов в задаче                                
                               reg_alpha=0, 
                               reg_lambda=1,
                               scale_pos_weight=1, 
                               seed=1234, 
                               silent=True, 
                               subsample=0.8
                                ),
                   fit_params={}, 
                   iid=True, 
                   n_jobs=-1,
                   param_grid={ #Изменяемые параметры
                               'min_child_weight': [1, 3, 5], 
                               'max_depth': [3, 5, 7]
                               'n_estimators': [100, 300, 500, 800, 1000],
                               'learning_rate': [0.05, 0.1, 0.3]
                              },
                   pre_dispatch='2*n_jobs', 
                   refit=True, 
                   scoring='accuracy', 
                   verbose=0
                  )

smart_xgboost.fit(X_train, y_train)

sorted(smart_xgboost.cv_results_.keys())

smart_xgboost.grid_scores_

In [ ]:
#Калибровка классификаторов. Нужна для рабочих моментов, чтобы лучше понять машину 
#Уточнение выданных машиной значений, с учетом "вероятности" попасть в класс
#Можно задать пороговые значения, которые определят класс. В промежутке между ними машина скажет "не знаю"
#Калибровка - это пересчет выходных значений, чтобы про них м.б. сказать - это Вероятность
#Изотоническая регрессия - это линия, у которой вектор не убывает
#Логистическая кривая, метод Платта - неубывающая линия, которая приблизит Pi к P


# Строим предсказание модели с возможностью посмотреть на вероятности принадлежать к классу
y_pred_train2 = model.predict_proba(X_train)
y_pred_test2 = model.predict_proba(X_test)

#Завершаем построенную машину калибровкой
model_sigmoid = CalibratedClassifierCV(model, 
                                       cv=2, 
                                       method='sigmoid' #Или "isotonic"
                                      )


#Обучаем калибровку на выборке валидации
model_sigmoid.fit(X_train, y_train)

#Смотрим на вероятности после калибровки
model_sigmoid.predict_proba(X_test)

# Предсказание класса для новых элементов
new_item = [1, 1, 1, 1]
model.predict([new_item])